In [7]:
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings
from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import login
from dotenv import load_dotenv
import os
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


# 코랩에서 실행 시 .env 설정
if os.path.exists("/content/drive/MyDrive/Colab Notebooks/LLM-KTAS/.env.txt"):
  try:
    os.rename("/content/drive/MyDrive/Colab Notebooks/LLM-KTAS/.env.txt", "/content/drive/MyDrive/Colab Notebooks/LLM-KTAS/.env")
  except FileNotFoundError:
    print("파일이 존재하지 않습니다.")
    exit(1)
  except OSError as e:
    print(f"파일 이름 변경 중 오류 발생: {e}")
    exit(1)

    load_dotenv('/content/drive/MyDrive/Colab Notebooks/LLM-KTAS/.env')

else:
  load_dotenv()

HF_API_TOKEN = os.environ["HF_API_TOKEN"]

In [8]:
# 임베딩 모델

hf_embeddings = HuggingFaceEndpointEmbeddings(
    model="intfloat/multilingual-e5-large-instruct",
    # task = 'feature-extraction',
    huggingfacehub_api_token=HF_API_TOKEN
)


# 벡터db (FAISS)
db = FAISS.load_local('db/KTAS_adult_faiss', hf_embeddings, allow_dangerous_deserialization=True)

In [9]:
# retrieval 결과에서 1, 2차 고려사항 추출 함수
def get_considerations(search_result):
    metadata = search_result[0][0].metadata
    primary = metadata['primary']
    secondary = metadata['secondary']

    return primary, secondary

In [37]:
# LLM 모델
llm = ChatOllama(model = "llama3.1:8b-instruct-q4_K_M")

# output parser
output_parser = StrOutputParser()

In [38]:
# 엔티티 추출 파이프라인

template_NER = """You are a specialized assistance for recognizing medical entities. You will receive a patient’s <Chief Coplaint> and <Additional Notes> and you must extract all entities that correspond to the chief complaint or primary symptoms.

Important Constraints:
- If multiple entities are present, output the in a single line separated by commas.
- Make sure to retain the exact text as it appears in the original note without any modifications.
- Output only the final result without adding any additional text, explanations, or formatting.

Read the following information carefully and extract all entities corresponding to the chief complaint or primary symptoms.

<Chief Complaint>:
{chief_complaint}

<Additional Notes>:
{additional_notes}"""

prompt_NER = PromptTemplate.from_template(template_NER)
chain_NER = prompt_NER | llm | output_parser

In [39]:
# 주증상 추출 파이프라인

template_symptom = """You are a specialized medical symptom classification assistant. You will receive a patient’s <Chief Coplaint> and <Additional Notes> and you must choose exactly one symptom from a predefined list of <Possible Symptoms>.

Important Constraints:
- You must choose only one symptom from the list.
- Your answer must exactly match the corresponding symptom in the <Possible Symptoms> (including spacing, punctuation, letter case, etc.).
- Output only the final result without adding any additional text, explanations, or formatting.

Read the following information carefully and determine which single symptom from the <Possible Symptoms> best matches the information given.

<Chief Complaint>:
{chief_complaint}

<Additional Notes>:
{additional_notes}

<Possible Symptoms>
{possible_symptoms}"""

prompt_symptom = PromptTemplate.from_template(template_symptom)
chain_symptom = prompt_symptom | llm | output_parser

In [40]:
# 주증상 추출 (self CoT) 파이프라인

template_symptom_self_CoT = """You are a specialized medical symptom classification assistant. You will receive a patient’s <Chief Coplaint> and <Additional Notes> and you must choose exactly one symptom from a predefined list of <Possible Symptoms>.

Important Constraints:
- You must choose only one symptom from the list.
- Your answer must exactly match the corresponding symptom in the <Possible Symptoms> (including spacing, punctuation, letter case, etc.).
- Think step by step.
- Output your reasoning step, but end with the final answer in the format of '<FINAL ANSWER: your answer>'


Read the following information carefully and determine which single symptom from the <Possible Symptoms> best matches the information given.

<Chief Complaint>:
{chief_complaint}

<Additional Notes>:
{additional_notes}

<Possible Symptoms>
{possible_symptoms}


Let's think step by step..."""

prompt_symptom_self_CoT = PromptTemplate.from_template(template_symptom_self_CoT)
chain_symptom_self_CoT = prompt_symptom_self_CoT | llm | output_parser

In [41]:
# 주증상 추출 (instruct CoT) 파이프라인

template_symptom_instruct_CoT = """You are a specialized medical symptom classification assistant. You will receive a patient’s <Chief Coplaint> and <Additional Notes> and you must choose exactly one symptom from a predefined list of <Possible Symptoms>.

Important Constraints:
- You must choose only one symptom from the list.
- Your answer must exactly match the corresponding symptom in the <Possible Symptoms> (including spacing, punctuation, letter case, etc.).
- Think step by step following the instruction below:
Step 1: Analyze <Chief Complaint> and <Additional Notes>
Step 2. Identify several potential candidates among the possible symptoms.
Step 3. Compare the candidates and select the most appropriate one as the final answer.
Step 4. Re-examine your final answer. If it seems inappropriate, go back to step 3 and refine your answer.
- Output your reasoning step, but end with the final answer in the format of '<FINAL ANSWER: your answer>'


Read the following information carefully and determine which single symptom from the <Possible Symptoms> best matches the information given.

<Chief Complaint>:
{chief_complaint}

<Additional Notes>:
{additional_notes}

<Possible Symptoms>
{possible_symptoms}


Let's think step by step..."""

prompt_symptom_instruct_CoT = PromptTemplate.from_template(template_symptom_instruct_CoT)
chain_symptom_instruct_CoT = prompt_symptom_instruct_CoT | llm | output_parser

In [42]:
# KTAS 증상 모음
with open ('../data/KTAS_symptoms.txt', 'r', encoding='utf-8-sig') as f:
    possible_symptoms = f.read()

possible_symptoms

'20주 미만의 임신\n20주 이상의 임신\n감각상실 / 이상감각\n감염 가능성 확인\n감염성 질환에 노출\n객혈\n경증 및 비특이적 증상 호소\n고혈당\n고혈압\n고환 통증 / 부종\n과다 복용\n과다호흡증후군\n관절 부종\n구토 / 구역\n국소성 부종 / 발적\n귀의 삼출물\n귀의 손상\n귀의 이물질\n기괴한 행동\n기억상실\n기침 / 코막힘\n기타 피부 상태\n눈 충혈 / 분비물\n눈 통증\n눈부심\n눈의 외상\n눈의 이물질\n눈의 화학물질 노출\n다뇨증\n다리 부기 / 부종\n단독 복부 외상 - 관통상\n단독 복부 외상 - 둔상\n단독 흉부 외상 - 관통상\n단독 흉부 외상 - 둔상\n동상 / 한랭손상\n두부 손상\n두통\n드레싱 교체\n딸꾹질\n떨림\n맥박이 없거나 차가운 사지\n목, 등, 허리 통증\n목의 부종 / 통증\n목의 외상\n몸통을 포함한 다발성 외상 - 관통상\n몸통을 포함한 다발성 외상 - 둔상\n물림\n물질 금단\n물질 오용 / 중독\n반지 제거\n발작\n발진\n변비\n보행 장애 / 운동 실조 / 강직\n복부 종괴 / 팽만\n복시\n복통\n불면증\n불안 / 위기 상황\n비정상 검사 결과\n사지 약화 / 뇌졸중 증상\n사회문제\n상기도감염 증상 호소\n상지 손상\n상지 통증\n상처 확인\n샅고랑 부위 통증 / 종괴\n생식기의 분비물 / 피부병변\n생식기의 외상\n석고 붕대 확인\n설사\n성폭행\n소변배출장애\n소양증\n수술 후 합병증\n숨참\n스테이플 / 봉합사 제거\n시력장애\n식욕부진\n실신 / 전실신\n심계항진 / 불규칙한 심장박동\n심정지(비외상성)\n심정지(외상성)\n쏘임\n안면 외상\n안면 통증 (비외상성 / 비치아성)\n안와주위 부종\n알레르기 반응\n연하장애 / 연하곤란\n열\n열상 / 천공\n영상 검사 / 검사실 검사\n옆구리 통증\n온열손상\n외상성 목, 등, 허리 손상\n외상없이 저절로 멍듦\n요로감염 증상\n우울증 / 자살 / 자해\n월경 문제\n유방의 발적 / 압통\n유해물질 흡입\n음경 부종\n음순 부종\n의료 

In [43]:
# 테스트 데이터

chief_complaint_1 = 'Disorientated'
additional_notes_1 = 'flapping tremor(-) 2차병원 내원하여 검사 안내함.'

chief_complaint_2 = 'Cough, Fever and chilling'
additional_notes_2 = '''fever upto 38.1

4pm 캐롤에프 해열제 목용 후 내원함.'''

chief_complaint_3 = 'AVF malfunction'
additional_notes_3 = '''월, 수, 금 HD > 금일 투석 못함

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



혈액검사하고 퇴원한 후 내일 시술 > 동의됨'''

chief_complaint_4 = 'Seizure like movement (zittering)'
additional_notes_4 = '''119도착 당시 bp 70대로 NS 500cc 가량 loading

5분 이내 upper eyeball deviation, 팔떨고 뒤로 넘어갔다고 함.'''

chief_complaint_5 = 'Head trauma'
additional_notes_5 = '''3/1 오른쪽 저리고 마비증상, 인지기능 저하 증상 발생 -> 현재 호전

타병원 hemorrhage 소견 받고 내원,

의뢰서 (-) 영상(+)

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



-------------------------------------------

현재 본원 응급실 의료진이 다수의 중증 응급환자 처지 중에 있어 본원 응급실에서는 즉각적인 응급 검사 및 처치가 불가능한 상태임을 설명하였으며 타 병원 방문하여 진료, 검사를 통해 진단 및 처치 진행해야 함을 고지하였음.

진단이 지연될 경우 미 확인된 의학적 상황으로 인한 환자의 상태 악화가 발생할 수 있음을 설명하였음. 환자 및 보호자 이와 같은 설명을 이해하였음을 확인함

-------------------------------------------'''

chief_complaint_6 = 'Myalgia'
additional_notes_6 = '''2월 중순부터 chilling (+) > local 에서 urine  lab 계속 진행

chilling, myalgia, fever +'''

chief_complaint_7 = 'Nausea and vomiting'
additional_notes_7 = '''2/26 타병원에서 Rib Fx. 진단받고, Painkiller PO med 복용 중 Nausea, Vomiting 있어 내원함. 어제부터 PO Painkiller 복용 중단했으나, 증상 지속된다고 함.

P.O.I [+]'''

chief_complaint_8 = 'Presyncope'
additional_notes_8 = '''보호자 진술 상 환자 쓰러졌다고 하나 의식소실 없고 정확히 쓰러진건지 모르겠다고 함.

신경학적 증상 -



이미 금일 2차병원에서 영상검사, 혈액검사 이상소견 없다고 들었다고 함.



------------------------------------------------------------------------------

현재 본원 응급실 의료진이 다수의 중증 응급환자 처지 중에 있어 본원 응급실에서는 즉각적인 응급 검사 및 처치가 불가능한 상태임을 설명하였으며 타 병원 방문하여 진료, 검사를 통해 진단 및 처치 진행해야 함을 고지하였음.

진단이 지연될 경우 미 확인된 의학적 상황으로 인한 환자의 상태 악화가 발생할 수 있음을 설명하였음. 환자 및 보호자 이와 같은 설명을 이해하였음을 확인함.'''

chief_complaint_9 = 'Anal bleeding, Dizziness, hematochezia'
additional_notes_9 = '''A.fib with RVR 심장내과협진후 퇴원하시분 lixiana 복용 후 소량의 혈변있어 내원함. 평소에 치질 있는 분으로 치질부위에 쓰라린 감있으며 그쪽에서 출혈이 있는것 같다고함. 2차병원 내원하여 검사 하고 복용약 지속여부는 외래에 문의 안내함.'''

chief_complaint_10 = 'Decreased visual acuity'
additional_notes_10 = '''왼쪽 안구 거의 전체가 회색으로 안개가 끼어있는 듯하다함.23년도 시신경염 증상과 비슷. 1-2시간 전부터 안구 통증 심해져서 내원'''

In [44]:
chief_complaint_list = [chief_complaint_1, chief_complaint_2, chief_complaint_3, chief_complaint_4, chief_complaint_5]
additional_notes_list = [additional_notes_1, additional_notes_2, additional_notes_3, additional_notes_4, additional_notes_5]

### 테스트1: 주호소와 비고를 그대로 사용하여 vector DB에서 semantic search (top 1)

In [30]:
for i in range(len(chief_complaint_list)):
    chief_complaint = chief_complaint_list[i]
    additional_notes = additional_notes_list[i]
    text = f"주호소: {chief_complaint}\n비고: {additional_notes}"

    search_result = db._similarity_search_with_relevance_scores(text, k=1)

    print('case ', i + 1)
    print(text, '\n')
    print('검색 결과: ', search_result[0][0].page_content, '\n\n')

case  1
주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함. 

검색 결과:  사지 약화 / 뇌졸중 증상 


case  2
주호소: Cough, Fever and chilling
비고: fever upto 38.1

4pm 캐롤에프 해열제 목용 후 내원함. 

검색 결과:  기침 / 코막힘 


case  3
주호소: AVF malfunction
비고: 월, 수, 금 HD > 금일 투석 못함

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



혈액검사하고 퇴원한 후 내일 시술 > 동의됨 

검색 결과:  심계항진 / 불규칙한 심장박동 


case  4
주호소: Seizure like movement (zittering)
비고: 119도착 당시 bp 70대로 NS 500cc 가량 loading

5분 이내 upper eyeball deviation, 팔떨고 뒤로 넘어갔다고 함. 

검색 결과:  사지 약화 / 뇌졸중 증상 


case  5
주호소: Head trauma
비고: 3/1 오른쪽 저리고 마비증상, 인지기능 저하 증상 발생 -> 현재 호전

타병원 hemorrhage 소견 받고 내원,

의뢰서 (-) 영상(+)

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



### 테스트2: 주호소와 비고를 그대로 사용하여 vector DB에서 semantic search (top k) -> LLM이 가장 적절한 1개 선택

In [31]:
for i in range(len(chief_complaint_list)):
    chief_complaint = chief_complaint_list[i]
    additional_notes = additional_notes_list[i]
    text = f"주호소: {chief_complaint}\n비고: {additional_notes}"

    search_result = db._similarity_search_with_relevance_scores(text, k=5)
    symptoms = [doc[0].page_content for doc in search_result]

    print('case ', i + 1)
    print(text, '\n')
    print('검색 결과: ', symptoms, '\n\n')

case  1
주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함. 

검색 결과:  ['사지 약화 / 뇌졸중 증상', '보행 장애 / 운동 실조 / 강직', '심정지(외상성)', '몸통을 포함한 다발성 외상 - 둔상', '심계항진 / 불규칙한 심장박동'] 


case  2
주호소: Cough, Fever and chilling
비고: fever upto 38.1

4pm 캐롤에프 해열제 목용 후 내원함. 

검색 결과:  ['기침 / 코막힘', '저체온증', '상기도감염 증상 호소', '처방전 / 투약 문의', '환자의 안녕에 대한 고려 (학대, 방임)'] 


case  3
주호소: AVF malfunction
비고: 월, 수, 금 HD > 금일 투석 못함

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



혈액검사하고 퇴원한 후 내일 시술 > 동의됨 

검색 결과:  ['심계항진 / 불규칙한 심장박동', '국소성 부종 / 발적', '환자의 안녕에 대한 고려 (학대, 방임)', '연하장애 / 연하곤란', '심정지(외상성)'] 


case  4
주호소: Seizure like movement (zittering)
비고: 119도착 당시 bp 70대로 NS 500cc 가량 loading

5분 이내 upper eyeball deviation, 팔떨고 뒤로 넘어갔다고 함. 

검색 결과:  ['사지 약화 / 뇌졸중 증상', '심정지(외상성)', '심정지(비외상성)', '떨림', '보행 장애 / 운동 실조 / 강직'] 


case  5
주호소: Head trauma
비고: 3/1 오른쪽 저리

### 테스트 3: LLM으로 주호소 추출 (NER) -> vector DB에서 증상 semantic search

In [45]:
for i in range(len(chief_complaint_list)):
    chief_complaint = chief_complaint_list[i]
    additional_notes = additional_notes_list[i]
    text = f"주호소: {chief_complaint}\n비고: {additional_notes}"

    entities = chain_NER.invoke({"chief_complaint": chief_complaint, "additional_notes": additional_notes})

    search_result = db._similarity_search_with_relevance_scores(text, k=5)
    symptoms = [doc[0].page_content for doc in search_result]

    print('case ', i + 1)
    print(text, '\n')
    print('엔티티: ', entities, '\n')
    print('검색 결과: ', symptoms, '\n\n')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


case  1
주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함. 

엔티티:  Disorientated, flapping tremor 

검색 결과:  ['사지 약화 / 뇌졸중 증상', '보행 장애 / 운동 실조 / 강직', '심정지(외상성)', '몸통을 포함한 다발성 외상 - 둔상', '심계항진 / 불규칙한 심장박동'] 




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


case  2
주호소: Cough, Fever and chilling
비고: fever upto 38.1

4pm 캐롤에프 해열제 목용 후 내원함. 

엔티티:  Cough, Fever, Chilling, Fever 

(Note: I've kept the exact text as it appears in the original note without any modifications) 

검색 결과:  ['기침 / 코막힘', '저체온증', '상기도감염 증상 호소', '처방전 / 투약 문의', '환자의 안녕에 대한 고려 (학대, 방임)'] 




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


case  3
주호소: AVF malfunction
비고: 월, 수, 금 HD > 금일 투석 못함

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



혈액검사하고 퇴원한 후 내일 시술 > 동의됨 

엔티티:  AVF malfunction 

검색 결과:  ['심계항진 / 불규칙한 심장박동', '국소성 부종 / 발적', '환자의 안녕에 대한 고려 (학대, 방임)', '연하장애 / 연하곤란', '심정지(외상성)'] 




/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


case  4
주호소: Seizure like movement (zittering)
비고: 119도착 당시 bp 70대로 NS 500cc 가량 loading

5분 이내 upper eyeball deviation, 팔떨고 뒤로 넘어갔다고 함. 

엔티티:  Seizure, zittering, palpitation is not present but "팔떨고" which means "shaking or trembling of the arms" could imply a seizure like movement, 70, NS 

Note: I did not find any information related to 'NS' in medical context. It might be an abbreviation used in the specific hospital or region that I am not aware of. 

검색 결과:  ['사지 약화 / 뇌졸중 증상', '심정지(외상성)', '심정지(비외상성)', '떨림', '보행 장애 / 운동 실조 / 강직'] 


case  5
주호소: Head trauma
비고: 3/1 오른쪽 저리고 마비증상, 인지기능 저하 증상 발생 -> 현재 호전

타병원 hemorrhage 소견 받고 내원,

의뢰서 (-) 영상(+)

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



-------------------------------------------

현재 본원 응급실 의료진이 다수의 중증 응급환자 처지 중에 있어 본원 응급실에서는 즉각적인 응급 검사 및 처치가 불가능한 상태임을 설명하였으며 타 병원 방문하여 진료

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


### 테스트 4: LLM으로 바로 주증상 선택

In [46]:
for i in range(len(chief_complaint_list)):
    chief_complaint = chief_complaint_list[i]
    additional_notes = additional_notes_list[i]
    text = f"주호소: {chief_complaint}\n비고: {additional_notes}"

    symptoms = chain_symptom.invoke({
        "chief_complaint": chief_complaint,
        "additional_notes": additional_notes,
        "possible_symptoms": possible_symptoms
    })

    print('case ', i + 1)
    print(text, '\n')
    print('LLM 추론 결과: ', symptoms, '\n\n')

case  1
주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함. 

LLM 추론 결과:  떨림 


case  2
주호소: Cough, Fever and chilling
비고: fever upto 38.1

4pm 캐롤에프 해열제 목용 후 내원함. 

LLM 추론 결과:  기침 / 코막힘 


case  3
주호소: AVF malfunction
비고: 월, 수, 금 HD > 금일 투석 못함

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



혈액검사하고 퇴원한 후 내일 시술 > 동의됨 

LLM 추론 결과:  AVF malfunction 


case  4
주호소: Seizure like movement (zittering)
비고: 119도착 당시 bp 70대로 NS 500cc 가량 loading

5분 이내 upper eyeball deviation, 팔떨고 뒤로 넘어갔다고 함. 

LLM 추론 결과:  떨림 


case  5
주호소: Head trauma
비고: 3/1 오른쪽 저리고 마비증상, 인지기능 저하 증상 발생 -> 현재 호전

타병원 hemorrhage 소견 받고 내원,

의뢰서 (-) 영상(+)

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



-------

### 테스트5: LLM으로 주호소 추출 (NER) → 주호소 참고하여 LLM으로 증상 선택

In [47]:
for i in range(len(chief_complaint_list)):
    chief_complaint = chief_complaint_list[i]
    additional_notes = additional_notes_list[i]
    text = f"주호소: {chief_complaint}\n비고: {additional_notes}"

    entities = chain_NER.invoke({"chief_complaint": chief_complaint, "additional_notes": additional_notes})

    symptoms = chain_symptom.invoke({
        "chief_complaint": entities,
        "additional_notes": 'None',
        "possible_symptoms": possible_symptoms
    })

    print('case ', i + 1)
    print(text, '\n')
    print('엔티티: ', entities, '\n')
    print('LLM 추론 결과: ', symptoms, '\n\n')

case  1
주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함. 

엔티티:  Disoriented, Flapping tremor 

LLM 추론 결과:  기괴한 행동 


case  2
주호소: Cough, Fever and chilling
비고: fever upto 38.1

4pm 캐롤에프 해열제 목용 후 내원함. 

엔티티:  Fever, Chilling, Cough 

LLM 추론 결과:  기침 / 코막힘 


case  3
주호소: AVF malfunction
비고: 월, 수, 금 HD > 금일 투석 못함

--------------------------------------------

■ 최근 21일 이내 해외 여행력 (국가명) : 없음

-------------------------------------------

■ 최근 7일 이내 COVID-19 확진력/증빙자료 여부 : 없음

-------------------------------------------



혈액검사하고 퇴원한 후 내일 시술 > 동의됨 

엔티티:  AVF malfunction, HD, 투석, 해외 여행력, COVID-19 

LLM 추론 결과:  고혈압 


case  4
주호소: Seizure like movement (zittering)
비고: 119도착 당시 bp 70대로 NS 500cc 가량 loading

5분 이내 upper eyeball deviation, 팔떨고 뒤로 넘어갔다고 함. 

엔티티:  Seizure, Zittering, BP (Blood Pressure) 70, Upper Eyeball Deviation, 팔떨기 (Palsy) 

LLM 추론 결과:  발작 


case  5
주호소: Head trauma
비고: 3/1 오른쪽 저리고 마비증상, 인지기능 저하 증상 발생 -> 현재 호전

타병원 hemorrhage 소견 받고 내원,

의뢰서 (-) 영상(+)

---------------

### 챗지피티 결과와 비교

case 1: 의식 수준의 변화  
case 2: 상기도감염 증상 호소  
case 3: 의료 장비 문제  
case 4: 발작  
case 5: 두부 손상

### 테스트6: LLM으로 주증상 선택 (알아서 CoT)

In [48]:
for i in range(len(chief_complaint_list)):
    chief_complaint = chief_complaint_list[i]
    additional_notes = additional_notes_list[i]
    text = f"주호소: {chief_complaint}\n비고: {additional_notes}"

    symptoms = chain_symptom_self_CoT.invoke({
        "chief_complaint": chief_complaint,
        "additional_notes": additional_notes,
        "possible_symptoms": possible_symptoms
    })

    print('case ', i + 1)
    print(text, '\n')
    print('LLM 추론 결과: ', symptoms, '\n\n')

case  1
주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함. 

LLM 추론 결과:  To determine the single symptom that best matches the given information, I will carefully analyze the provided details:

<Chief Complaint>: Disorientated
<Additional Notes>: flapping tremor(-) 2차병원 내원하여 검사 안내함.

From the <Chief Complaint>, it is clear that the patient is experiencing disorientation. The additional notes mention a "flapping tremor," which is often associated with neurological conditions, specifically Alzheimer's disease or other forms of dementia. However, given the presence of a flapping tremor, I will focus on symptoms related to this specific finding.

Step 1: Reviewing possible symptoms that match the description:
- The symptom "기괴한 행동" translates to "Abnormal behavior." This is not directly related to disorientation or the flapping tremor.
- However, another symptom stands out, "기억상실" which means "Memory loss," but this doesn't exactly match the provided symptoms.

Step 2: Re-examinin

### 테스트7: LLM으로 주증상 선택 (CoT 단계 지시)
#### 1. 후보 선택
#### 2. 최종 선택

In [49]:
for i in range(len(chief_complaint_list)):
    chief_complaint = chief_complaint_list[i]
    additional_notes = additional_notes_list[i]
    text = f"주호소: {chief_complaint}\n비고: {additional_notes}"

    symptoms = chain_symptom_instruct_CoT.invoke({
        "chief_complaint": chief_complaint,
        "additional_notes": additional_notes,
        "possible_symptoms": possible_symptoms
    })

    print('case ', i + 1)
    print(text, '\n')
    print('LLM 추론 결과: ', symptoms, '\n\n')

case  1
주호소: Disorientated
비고: flapping tremor(-) 2차병원 내원하여 검사 안내함. 

LLM 추론 결과:  To determine the best-matching symptom from the <Possible Symptoms>, I will follow the steps outlined:

**Step 1: Analyze Chief Complaint and Additional Notes**

The patient's **Chief Complaint** is "Disorientated" which implies a problem with their sense of direction or orientation. The **Additional Notes** mention a flapping tremor, which suggests a neurological issue. 

**Step 2: Identify several potential candidates among the possible symptoms**

Given the information from Step 1, several symptoms stand out as potential matches:
- 기괴한 행동 (Abnormal behavior)
- 기억상실 (Memory loss)
- 기괴한 행동 seems to directly relate to disorientation and abnormal behavior, which could be inferred from a flapping tremor.

**Step 3: Compare the candidates and select the most appropriate one as the final answer**

Comparing these symptoms:
- 기괴한 행동 (Abnormal behavior) matches both the disorientation and the unusual movement m

### 테스트8: LLM으로 주증상 후보 선택 -> 해당 주증상들에 대한 설명 또는 예시 retrieve -> Few-shot으로 최종 주증상 선택

### 테스트9: LLM으로 주증상 후보 선택 (CoT) -> 해당 주증상들에 대한 설명 또는 예시 retrieve -> Few-shot으로 최종 주증상 선택 (CoT)